In [108]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("Eval")

LangSmith 추적을 시작합니다.
[프로젝트명]
Eval


In [46]:
import pandas as pd

In [59]:
df = pd.read_csv("outputs/synthetic_out.csv")
df

,Unnamed: 0,query,answers,evidence,page_number
0,0,What are three factual statements about the pu...,1) The work is published under the responsibil...,This work is published under the responsibilit...,4
1,1,Which entity does Turkiye recognise as the Tur...,Turkiye recognises the Turkish Republic of Nor...,Note by the Republic of Turkiye\n\nThe informa...,4
2,2,What is the ISBN for the PDF version of the OE...,ISBN 978-92-64-57606-3 (PDF),content : ISBN 978-92-64-57606-3 (PDF) \nISBN...,4
3,3,List three licensing and arbitration condition...,1) It is an adaptation of an original work by ...,Adaptations- you must cite the original work a...,4
4,4,What are three atomic facts that can be extrac...,The page includes the header 'Table of content...,I 3\n\nI 3\n \n\nTable of contents\n=========...,5
...,...,...,...,...,...
102,102,What are the 3 2050 targets described in the d...,"- By 2050, governance in G20 emerging-market e...","economies by 2050, 2) improvements in governan...",23
103,103,What is the title of the document on page 23?,1) Title: FINDING THE RIGHT BALANCE IN UNCERTA...,FINDING THE RIGHT BALANCE IN UNCERTAIN TIMES ©...,23
104,104,What are three key facts shown on page 24 of t...,Fact 1: Document title is OECD Economic Outloo...,"OECD Economic Outlook, Interim Report\n=======...",24
105,105,What is expected for global growth after the f...,Global growth was more resilient than anticipa...,Global growth is expected to moderate as the f...,24


In [70]:
df = pd.read_csv("outputs/synthetic_out.csv")
df

,Unnamed: 0,query,answers,evidence,page_number
0,0,What are three factual statements about the pu...,1) The work is published under the responsibil...,This work is published under the responsibilit...,4
1,1,Which entity does Turkiye recognise as the Tur...,Turkiye recognises the Turkish Republic of Nor...,Note by the Republic of Turkiye\n\nThe informa...,4
2,2,What is the ISBN for the PDF version of the OE...,ISBN 978-92-64-57606-3 (PDF),content : ISBN 978-92-64-57606-3 (PDF) \nISBN...,4
3,3,List three licensing and arbitration condition...,1) It is an adaptation of an original work by ...,Adaptations- you must cite the original work a...,4
4,4,What are three atomic facts that can be extrac...,The page includes the header 'Table of content...,I 3\n\nI 3\n \n\nTable of contents\n=========...,5
...,...,...,...,...,...
102,102,What are the 3 2050 targets described in the d...,"- By 2050, governance in G20 emerging-market e...","economies by 2050, 2) improvements in governan...",23
103,103,What is the title of the document on page 23?,1) Title: FINDING THE RIGHT BALANCE IN UNCERTA...,FINDING THE RIGHT BALANCE IN UNCERTAIN TIMES ©...,23
104,104,What are three key facts shown on page 24 of t...,Fact 1: Document title is OECD Economic Outloo...,"OECD Economic Outlook, Interim Report\n=======...",24
105,105,What is expected for global growth after the f...,Global growth was more resilient than anticipa...,Global growth is expected to moderate as the f...,24


In [80]:
df.head()

,Unnamed: 0,query,answers,evidence,page_number
0,0,What are three factual statements about the pu...,1) The work is published under the responsibil...,This work is published under the responsibilit...,4
1,1,Which entity does Turkiye recognise as the Tur...,Turkiye recognises the Turkish Republic of Nor...,Note by the Republic of Turkiye\n\nThe informa...,4
2,2,What is the ISBN for the PDF version of the OE...,ISBN 978-92-64-57606-3 (PDF),content : ISBN 978-92-64-57606-3 (PDF) \nISBN...,4
3,3,List three licensing and arbitration condition...,1) It is an adaptation of an original work by ...,Adaptations- you must cite the original work a...,4
4,4,What are three atomic facts that can be extrac...,The page includes the header 'Table of content...,I 3\n\nI 3\n \n\nTable of contents\n=========...,5


In [78]:
from langsmith import Client

client = Client()


In [82]:
input_keys = ['query'] # replace with your input column names
output_keys = ['answers', 'page_number'] # replace with your output column names
dataset_name = "My Parquet Dataset - 2"
dataset = client.upload_dataframe(
    df=df,
    input_keys=input_keys,
    output_keys=output_keys,
    name=dataset_name,
    description="Dataset created from a parquet file",
    data_type="kv" # The default
)

In [10]:
import pickle
from langchain.schema import Document
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

In [18]:
with open("outputs/split_documents.pkl", "rb") as f:
    split_documents = pickle.load(f)

In [88]:
from typing import Annotated, TypedDict
from langgraph.graph import END, StateGraph
from langgraph.checkpoint.memory import MemorySaver

vectorstore = FAISS.load_local(
    "faiss_index", 
    OpenAIEmbeddings(),
    allow_dangerous_deserialization=True  # needed in newer versions
)
faiss_retriever = vectorstore.as_retriever(search_kwargs={"k": 1})
bm25_retriever = BM25Retriever.from_documents(
    split_documents,
)
bm25_retriever.k = 1
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever],
    weights=[0.7, 0.3],
)

# GraphState 상태 정의
class GraphState(TypedDict):
    question: Annotated[str, "Question"]  # 질문
    context: Annotated[str, "Context"]  # 문서의 검색 결과
    answer: Annotated[str, "Answer"]  # 답변
    documents: Annotated[list[Document], "Documents"]  # 검색된 문서
    page_number: Annotated[list[int], "Page Number"]  # 페이지 번호
    
# 문서 검색 노드
def retrieve_document(state: GraphState) -> GraphState:
    # 질문을 상태에서 가져옵니다.
    latest_question = state["question"]

    # 문서에서 검색하여 관련성 있는 문서를 찾습니다.
    retrieved_docs = ensemble_retriever.invoke(latest_question)
    context = "".join([doc.page_content for doc in retrieved_docs])
    # 검색된 문서를 context 키에 저장합니다.
    return {"documents": retrieved_docs, "context": context}


# 답변 생성 노드
def llm_answer(state: GraphState) -> GraphState:
    # 질문을 상태에서 가져옵니다.
    latest_question = state["question"]

    # 검색된 문서를 상태에서 가져옵니다.
    context = state["context"]
   
    llm = ChatOpenAI(model_name="gpt-5-mini", temperature=0)
      
    system_prompt = """You are an assistant for question-answering tasks. 
        Use the following pieces of retrieved context to answer the question. 
        If you don't know the answer, just say that you don't know. 
        
    """
    
    prompt = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": "Context: " + context},
        {"role": "user", "content": "Question: " + latest_question},
    ]
        
    # 체인을 호출하여 답변을 생성합니다.
    response = llm.invoke(prompt)
    page_number = []
    for doc in state["documents"]:
        page_number.append(doc.metadata["page"])
    # 생성된 답변, (유저의 질문, 답변) 메시지를 상태에 저장합니다.
    return {"answer": response.content, "documents": state["documents"], "page_number": page_number}


# 그래프 생성
workflow = StateGraph(GraphState)

# 노드 정의
workflow.add_node("retrieve", retrieve_document)
workflow.add_node("llm_answer", llm_answer)

# 엣지 정의
workflow.add_edge("retrieve", "llm_answer")  # 검색 -> 답변
workflow.add_edge("llm_answer", END)  # 답변 -> 종료

# 그래프 진입점 설정
workflow.set_entry_point("retrieve")

# 체크포인터 설정
memory = MemorySaver()

# 컴파일
app = workflow.compile(checkpointer=memory)

In [20]:
from langchain_core.runnables import RunnableConfig
from langchain_teddynote.messages import invoke_graph, stream_graph, random_uuid

# config 설정(재귀 최대 횟수, thread_id)
config = RunnableConfig(recursion_limit=20, configurable={"thread_id": random_uuid()})

# 질문 입력
inputs = GraphState(question="터키가 직면한 위기와 도전을 알려주세요.")

In [9]:
inputs

{'question': '터키가 직면한 위기와 도전을 알려주세요.'}

In [52]:
result = app.invoke(inputs, config)

In [53]:
result['answer'], result['documents'], result['page_number']

('제공된 문맥을 기준으로 보면, 터키가 직면한 주요 위기와 도전은 다음과 같습니다.\n\n- 거시경제·통화정책 관련 부담  \n  - 물가(인플레이션) 관리 필요: 중앙은행은 경계 태세를 유지해야 하며, 기저 인플레이션이 목표치 쪽으로 완화되고 물가 기대가 잘 고정될 경우에만 금리 인하를 고려할 수 있음.  \n  - 재정 지속가능성 문제: 장기적 부채 지속가능성을 지키고 향후 충격에 대응할 수 있도록 재정기조의 규율(긴축)이 필요함.\n\n- 구조개혁과 생산성 향상 과제  \n  - 생활수준을 지속적으로 개선하고 신기술(예: 인공지능)의 잠재력을 실현하기 위해 강화된 구조개혁이 요구됨.\n\n- 무역·국제협력 관련 도전  \n  - 글로벌 무역체계 내에서 협력적 참여와 무역정책의 투명성·예측가능성 제고가 필요함.\n\n- 외교·정치적 현안 (키프로스 문제)  \n  - 문서에 따르면 키프로스 문제에서 터키는 북키프로스(Turkish Republic of Northern Cyprus)를 인정하고 있으며, 섬의 남부(Republic of Cyprus)에 대한 국제적 인정 문제로 인해 지속적인 입장 표명이 존재함. 이는 터키가 직면한 외교적·지역적 도전의 하나임.\n\n문맥은 일반적 권고와 입장 표명을 담고 있어 구체적 수치나 최신 상황(예: 현재 인플레이션률, 재정적자 규모 등)은 제공되지 않습니다. 더 구체적인 최신 정보나 통계(예: 터키의 최근 물가·재정지표, 키프로스 관련 외교 현황)를 원하시면 알려주세요.',
 [Document(metadata={'page': 24, 'image_id': [], 'image_path': [], 'text_summary': [], 'image_summary': [], 'id': 108}, page_content='The Interim Report says that countries need to find ways of engaging co‑operatively within the global trading system  

In [16]:
def rag_bot(question: str) -> dict:
    inputs = GraphState(question=question)
    res = app.invoke(inputs, config)
    return {"answer": res['answer'], "documents": res['documents']}

In [11]:
from typing_extensions import Annotated, TypedDict

# Grade output schema
class CorrectnessGrade(TypedDict):
    # Note that the order in the fields are defined is the order in which the model will generate them.
    # It is useful to put explanations before responses because it forces the model to think through
    # its final response before generating it:
    explanation: Annotated[str, ..., "Explain your reasoning for the score"]
    correct: Annotated[bool, ..., "True if the answer is correct, False otherwise."]

# Grade prompt
correctness_instructions = """You are a teacher grading a quiz. You will be given a QUESTION, the GROUND TRUTH (correct) ANSWER, and the STUDENT ANSWER. Here is the grade criteria to follow:
(1) Grade the student answers based ONLY on their factual accuracy relative to the ground truth answer. (2) Ensure that the student answer does not contain any conflicting statements.
(3) It is OK if the student answer contains more information than the ground truth answer, as long as it is factually accurate relative to the  ground truth answer.

Correctness:
A correctness value of True means that the student's answer meets all of the criteria.
A correctness value of False means that the student's answer does not meet all of the criteria.

Explain your reasoning in a step-by-step manner to ensure your reasoning and conclusion are correct. Avoid simply stating the correct answer at the outset."""

# Grader LLM
grader_llm = ChatOpenAI(model="gpt-5-nano", temperature=0).with_structured_output(
    CorrectnessGrade, method="json_schema", strict=True
)

def correctness(inputs: dict, outputs: dict, reference_outputs: dict) -> bool:
    """An evaluator for RAG answer accuracy"""
    answers = f"""\
QUESTION: {inputs['question']}
GROUND TRUTH ANSWER: {reference_outputs['answer']}
STUDENT ANSWER: {outputs['answer']}"""
    # Run evaluator
    grade = grader_llm.invoke([
        {"role": "system", "content": correctness_instructions},
        {"role": "user", "content": answers}
    ])
    return grade["correct"]

In [12]:
# Grade output schema
class RelevanceGrade(TypedDict):
    explanation: Annotated[str, ..., "Explain your reasoning for the score"]
    relevant: Annotated[
        bool, ..., "Provide the score on whether the answer addresses the question"
    ]

# Grade prompt
relevance_instructions = """You are a teacher grading a quiz. You will be given a QUESTION and a STUDENT ANSWER. Here is the grade criteria to follow:
(1) Ensure the STUDENT ANSWER is concise and relevant to the QUESTION
(2) Ensure the STUDENT ANSWER helps to answer the QUESTION

Relevance:
A relevance value of True means that the student's answer meets all of the criteria.
A relevance value of False means that the student's answer does not meet all of the criteria.

Explain your reasoning in a step-by-step manner to ensure your reasoning and conclusion are correct. Avoid simply stating the correct answer at the outset."""

# Grader LLM
relevance_llm = ChatOpenAI(model="gpt-5-nano", temperature=0).with_structured_output(
    RelevanceGrade, method="json_schema", strict=True
)

# Evaluator
def relevance(inputs: dict, outputs: dict) -> bool:
    """A simple evaluator for RAG answer helpfulness."""
    answer = f"QUESTION: {inputs['question']}\nSTUDENT ANSWER: {outputs['answer']}"
    grade = relevance_llm.invoke([
        {"role": "system", "content": relevance_instructions},
        {"role": "user", "content": answer}
    ])
    return grade["relevant"]

In [13]:
# Grade output schema
class GroundedGrade(TypedDict):
    explanation: Annotated[str, ..., "Explain your reasoning for the score"]
    grounded: Annotated[
        bool, ..., "Provide the score on if the answer hallucinates from the documents"
    ]

# Grade prompt
grounded_instructions = """You are a teacher grading a quiz. You will be given FACTS and a STUDENT ANSWER. Here is the grade criteria to follow:
(1) Ensure the STUDENT ANSWER is grounded in the FACTS. (2) Ensure the STUDENT ANSWER does not contain "hallucinated" information outside the scope of the FACTS.

Grounded:
A grounded value of True means that the student's answer meets all of the criteria.
A grounded value of False means that the student's answer does not meet all of the criteria.

Explain your reasoning in a step-by-step manner to ensure your reasoning and conclusion are correct. Avoid simply stating the correct answer at the outset."""

# Grader LLM
grounded_llm = ChatOpenAI(model="gpt-5-nano", temperature=0).with_structured_output(
    GroundedGrade, method="json_schema", strict=True
)

# Evaluator
def groundedness(inputs: dict, outputs: dict) -> bool:
    """A simple evaluator for RAG answer groundedness."""
    doc_string = "\n\n".join(doc.page_content for doc in outputs["documents"])
    answer = f"FACTS: {doc_string}\nSTUDENT ANSWER: {outputs['answer']}"
    grade = grounded_llm.invoke([
        {"role": "system", "content": grounded_instructions},
        {"role": "user", "content": answer}
    ])
    return grade["grounded"]

In [14]:
# Grade output schema
class RetrievalRelevanceGrade(TypedDict):
    explanation: Annotated[str, ..., "Explain your reasoning for the score"]
    relevant: Annotated[
        bool,
        ...,
        "True if the retrieved documents are relevant to the question, False otherwise",
    ]

# Grade prompt
retrieval_relevance_instructions = """You are a teacher grading a quiz. You will be given a QUESTION and a set of FACTS provided by the student. Here is the grade criteria to follow:
(1) You goal is to identify FACTS that are completely unrelated to the QUESTION
(2) If the facts contain ANY keywords or semantic meaning related to the question, consider them relevant
(3) It is OK if the facts have SOME information that is unrelated to the question as long as (2) is met

Relevance:
A relevance value of True means that the FACTS contain ANY keywords or semantic meaning related to the QUESTION and are therefore relevant.
A relevance value of False means that the FACTS are completely unrelated to the QUESTION.

Explain your reasoning in a step-by-step manner to ensure your reasoning and conclusion are correct. Avoid simply stating the correct answer at the outset."""

# Grader LLM
retrieval_relevance_llm = ChatOpenAI(
    model="gpt-5-nano", temperature=0
).with_structured_output(RetrievalRelevanceGrade, method="json_schema", strict=True)

def retrieval_relevance(inputs: dict, outputs: dict) -> bool:
    """An evaluator for document relevance"""
    doc_string = "\n\n".join(doc.page_content for doc in outputs["documents"])
    answer = f"FACTS: {doc_string}\nQUESTION: {inputs['question']}"
    # Run evaluator
    grade = retrieval_relevance_llm.invoke([
        {"role": "system", "content": retrieval_relevance_instructions},
        {"role": "user", "content": answer}
    ])
    return grade["relevant"]

In [21]:
res = rag_bot("터키가 직면한 위기와 도전을 알려주세요.")
res['answer'], res['documents']

response content='제공된 OECD 중간보고서(2025년 9월)에 따르면 문서에서 언급된 터키(Republic of Turkiye)가 직면한 주요 위기·도전은 다음과 같습니다.\n\n- 대외무역과 국제협력의 필요성: 세계 무역체제 내에서 다른 나라들과 협력적으로 관여하고 무역정책의 투명성과 예측가능성을 높여야 함.  \n- 물가·통화 정책의 긴축·완화 판단: 근원 인플레이션이 목표로 완화될 것으로 예상될 때 인플레이션 기대가 잘 고정돼 있는 한 정책금리를 낮출 수 있으나, 중앙은행은 계속 경계를 늦추지 말아야 함(물가안정 리스크).  \n- 재정 지속가능성: 장기 채무 지속가능성을 지키고 향후 충격에 대응할 수 있도록 재정 규율을 유지할 필요가 있음.  \n- 구조개혁 필요성: 생활수준을 지속적으로 개선하고 인공지능 등 신기술에서 잠재력을 실현하기 위해 보다 강한 구조개혁 노력이 요구됨.  \n- 외교·영토 문제(사이프러스): 문서에는 사이프러스 관련 각주가 있어 터키가 북키프로스(TRNC)를 인정하고 있으며, 섬의 분단 문제와 관련한 외교적 입장·도전이 있음을 명시함.\n\n참고: 이 답변은 제공된 문서 내용에 근거한 요약입니다. 보다 구체적(예: 인플레이션율, 재정적자 규모, 실업률 등)인 경제지표나 최근 사건 기반 분석을 원하시면 추가 자료를 알려주시면 더 자세히 설명해 드리겠습니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 966, 'prompt_tokens': 459, 'total_tokens': 1425, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 576, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_t

('제공된 OECD 중간보고서(2025년 9월)에 따르면 문서에서 언급된 터키(Republic of Turkiye)가 직면한 주요 위기·도전은 다음과 같습니다.\n\n- 대외무역과 국제협력의 필요성: 세계 무역체제 내에서 다른 나라들과 협력적으로 관여하고 무역정책의 투명성과 예측가능성을 높여야 함.  \n- 물가·통화 정책의 긴축·완화 판단: 근원 인플레이션이 목표로 완화될 것으로 예상될 때 인플레이션 기대가 잘 고정돼 있는 한 정책금리를 낮출 수 있으나, 중앙은행은 계속 경계를 늦추지 말아야 함(물가안정 리스크).  \n- 재정 지속가능성: 장기 채무 지속가능성을 지키고 향후 충격에 대응할 수 있도록 재정 규율을 유지할 필요가 있음.  \n- 구조개혁 필요성: 생활수준을 지속적으로 개선하고 인공지능 등 신기술에서 잠재력을 실현하기 위해 보다 강한 구조개혁 노력이 요구됨.  \n- 외교·영토 문제(사이프러스): 문서에는 사이프러스 관련 각주가 있어 터키가 북키프로스(TRNC)를 인정하고 있으며, 섬의 분단 문제와 관련한 외교적 입장·도전이 있음을 명시함.\n\n참고: 이 답변은 제공된 문서 내용에 근거한 요약입니다. 보다 구체적(예: 인플레이션율, 재정적자 규모, 실업률 등)인 경제지표나 최근 사건 기반 분석을 원하시면 추가 자료를 알려주시면 더 자세히 설명해 드리겠습니다.',
 [Document(metadata={'page': 24, 'image_id': [], 'image_path': [], 'text_summary': [], 'image_summary': [], 'id': 108}, page_content='The Interim Report says that countries need to find ways of engaging co‑operatively within the global trading system  \nand working together to make trade policy more transparent and predictable. C

In [109]:
from utils.embed_openai import get_openai_embedding, calculate_cosine_similarity, EMBEDDING_MODEL

In [111]:
def f1_score_summary_evaluator(outputs: list[dict], reference_outputs: list[dict]) -> dict:
    true_positives = 0
    false_positives = 0
    false_negatives = 0

    for output_dict, reference_output_dict in zip(outputs, reference_outputs):
        print(output_dict, reference_output_dict)
        
        output_answer = output_dict["answer"]
        reference_output_answer = reference_output_dict["answers"]

        output_page_number = output_dict["page_number"]
        reference_output_page_number = reference_output_dict["page_number"]

        # TODO: 문장 비교 방법 추가(ex 코사인 유사도도)
        output = get_openai_embedding(output_answer)
        reference_output = get_openai_embedding(reference_output_answer)
        similarity = calculate_cosine_similarity(output, reference_output)

        if similarity > 0.6:
            true_positives += 1

        elif output_answer != reference_output_answer:
            false_positives += 1
        elif isinstance(output_page_number, (list, tuple, set)):
                if reference_output_page_number in output_page_number:
                    true_positives += 1
                else:
                    false_negatives += 1
        
    if true_positives == 0:
        return {"key": "f1_score", "score": 0.0}

    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / (true_positives + false_negatives)

    f1_score = 2 * (precision * recall) / (precision + recall)

    return {"key": "f1_score", "score": f1_score} 
   
    

In [28]:
def rag_bot_batch(question: list[str]) -> dict:
    inputs = [GraphState(question=q) for q in question]
    res = app.batch(inputs, config)
    outputs = [{'answer': r['answer'], 'documents': r['documents']} for r in res]
    return outputs

In [56]:
def rag_bot_invoke(question: str) -> dict:
    inputs = GraphState(question=question)
    res = app.invoke(inputs, config)
    return {'answer': res['answer'], 'documents': res['documents'], 'page_number': res['page_number']}


In [40]:
def correct(outputs: dict, reference_outputs: dict) -> bool:
    """Row-level correctness evaluator."""
    return outputs["answer"] == reference_outputs["answer"]

In [112]:
def target(inputs: dict) -> dict:
    return rag_bot_invoke(inputs["query"])
dataset_name = "My Parquet Dataset"
examples = client.list_examples(dataset_name=dataset_name, limit=10)

experiment_results = client.evaluate(
    target,
    data=examples,
    # evaluators=[correct],
    summary_evaluators=[f1_score_summary_evaluator],
    experiment_prefix="rag-f1",
    metadata={"version": "langgraph, gpt-5-nano"},
)


View the evaluation results for experiment: 'rag-f1-403e52d3' at:
https://smith.langchain.com/o/824afa80-3494-5b99-80b3-21be59fa9f24/datasets/f9df77b5-13a8-40a2-8fc9-efdaf4e504bf/compare?selectedSessions=4c179022-898c-43e7-83b1-c617844fde9d




0it [00:00, ?it/s]

{'answer': 'Recommended actions are to continue developing and updating crypto‑asset regulation, using tailored domestic approaches where appropriate but designed to be technologically neutral so they don’t distort competition. Jurisdictions should have flexibility to implement their own rules, but because crypto‑assets move easily across borders, strong international regulatory cooperation ( coordination on standards, information‑sharing and cross‑border enforcement ) is essential.', 'documents': [Document(metadata={'page': 21, 'image_id': [168], 'image_path': ['/images/sample_cropped_chart_168.png'], 'text_summary': [], 'image_summary': 'Here’s a description of the image and what it shows.\n\nWhat the image is\n- Figure 10: “Policy rates are expected to ease gradually.”\n- It contains two line charts side by side:\n  - Panel A: Advanced economies\n  - Panel B: Emerging-market economies\n- Each panel shows policy interest rates (percent) over time, from about 2022 through 2026.\n- Col

In [91]:
# Explore results locally as a dataframe if you have pandas installed
experiment_results.to_pandas()

,inputs.query,outputs.answer,outputs.documents,outputs.page_number,error,reference.answers,reference.page_number,execution_time,example_id,id
0,What regulatory actions and international coop...,- Continue developing and updating regulations...,[page_content='should continue efforts to deve...,[21],None,There should be continued efforts to develop a...,21,5.589583,0860272e-d73f-451c-81a6-2d75c599e873,2cfc4eff-f455-450d-a363-48dccf4416a7
1,List three atomic facts from the Interim EO pr...,- Turkiye's interim EO projection for 2024 is ...,[page_content='I 7\n\nI 7\n\nTable 3. Core inf...,"[9, 7]",None,Canada's 2024 Interim EO projection is 2.4; Ge...,8,14.949568,0abe52de-473c-4ced-8224-fed22d567119,cde50b01-30f1-4938-b0b3-de8a24c45ec4
2,What are three facts about global merchandise ...,- Global merchandise trade grew rapidly in the...,[page_content='2. Global merchandise trade gre...,[10],None,Global merchandise trade grew rapidly in the f...,10,11.005207,0c076399-2a6f-4d44-b9f5-5d47a3e99fbe,d9f12685-8133-481d-9413-48db6717e886
3,What factors behind the slowdown in global gro...,The OECD Economic Outlook highlights a sustain...,[page_content='32. Recent editions of the OECD...,[22],None,1) Weak investment is a factor behind the slow...,22,3.599108,0df19e50-073b-4574-807f-8ac908d51e0c,37cab953-4d7a-4028-a0d2-dc4fad03f4ed
4,What are three GDP growth figures shown on pag...,"Three GDP growth figures shown are: 4%, 3.5%, ...",[page_content='Note: Panel A is based on month...,"[14, 10]",None,Q2 2023 GDP growth: 4.5%; Q4 2023 GDP growth: ...,10,3.726131,0e8971b6-831f-4605-b6d4-b56ae535c4a4,5130af06-07c0-4b07-93e9-49ad3b9fa755
5,What three indicators are cited as signs of he...,The text cites: (1) a historically high estima...,"[page_content='12. Despite these changes, some...",[16],None,Historically-high estimated real term premium ...,16,6.114404,12897468-08f5-4942-8620-758aa23f9169,10fda37b-a398-4036-b362-210fdc3eedc9
6,What is the projected global potential output ...,The projected global potential output growth i...,[page_content='31. At a time of limited fiscal...,"[22, 22]",None,2¼ per cent,22,3.651334,13b8357b-7b5e-4bae-bd40-22e571fd08d1,dae1b044-6504-43da-988e-bf0193550b59
7,What are three policy-rate actions described i...,"From the excerpt you gave, three policy-rate a...",[page_content='Note: The three panels show yea...,"[15, 21]",None,1) The United States and Canada reduced policy...,20,12.093793,1a34955f-634e-49da-bb05-850471bc6d00,49bffb74-94f3-4697-a4b6-1ebcadbcc22d
8,What are the values for TUR in the first chart...,TUR (first chart): 100%\n\nARG (second chart):...,"[page_content='Note: In Panels A and B, 2025 e...","[13, 15]",None,TUR in the first chart = 100%; ARG in the seco...,15,5.791001,1b447d9f-279c-47d5-9790-6d40b9e6f65f,6ba78c6f-fa69-49b5-a447-17da46a4d053
9,List three factual statements about debt susta...,1. Credible and well-designed budgetary measur...,[page_content='Credible and well-designed budg...,[21],None,1) Credible and well-designed budgetary measur...,21,8.070977,1cd1ace1-32f1-4759-992b-ba22b9862ea6,0d6f2092-020f-4c5b-8b07-f75fd7371410
